In [1]:
pip install ucimlrepo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [161]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset from uci repo using the ucirepo packages 
cancerData = fetch_ucirepo(id=17) 
  
# data retrieved as dataframes whe using ucimlrepo
X = cancerData.data.features
y = cancerData.data.targets.replace({'M': 1, 'B': 0})
y=y.squeeze()

print(y.shape)
print(X.shape)

(569,)
(569, 30)


In [197]:
from sklearn.model_selection import train_test_split



def modelsig(X,theta):
    dotProd = np.dot(X,theta)
    return 1/(1+np.exp(-dotProd))


def newtonRaphson(X,y):
    #initailze the theta with 0 with the shape of X
    theta=np.zeros(X.shape[1])
    l=0
    while l<100:
        #compute gradient
        mu=modelsig(X,theta)
        l+=1
        grad = np.dot(X.T,mu - y) 
        #compute hessian H=tranpose(X)SX
        #where S= diag(mu(1-mu))
        S=np.diag(mu * (1-mu))
        #adding the regularisation term to the hessian matrix so as it does not become singular before convergence
        hessian = np.dot(X.T, np.dot(S, X)) + 1e-1 * np.eye(X.shape[1])
        theta= theta - np.dot(np.linalg.pinv(hessian),grad)   
        #check for convergence and if reached break the loop
        if np.linalg.norm(np.dot(np.linalg.pinv(hessian), grad)) < 1e-5:
            break
        
    return theta



accuracylist=[]
#Given to do 10 splits on data set and calculate average 
for i in range(0,10):
    Xtrain,Xtest,ytrain,ytest= train_test_split(X,y,test_size=0.2,random_state=None)
    theta = newtonRaphson(Xtrain.to_numpy(),ytrain.to_numpy())
    
    #predict with the test data
    ypred=modelsig(Xtest.to_numpy(),theta)
    
    #created an array to calculate values that are above the threshold
    prr=[]
    for ea in ypred:
        if ea>=0.5:
            prr.append(1)
        else:
            prr.append(0)
    prr=np.array(prr)
    # Calculate accuracy of the predicted data built using the model and theta
    accuracy = np.mean(prr == ytest)
    accuracylist.append(accuracy)
    print(f"Accuracy at fold {i+1}: {accuracy}")
    
avgAccuracy=0
for eachAccu in accuracylist:
    avgAccuracy+=eachAccu

print("Average Accuracy = ",avgAccuracy/10)
    
    

Accuracy at fold 1: 0.9736842105263158
Accuracy at fold 2: 0.956140350877193
Accuracy at fold 3: 0.9736842105263158
Accuracy at fold 4: 0.9649122807017544
Accuracy at fold 5: 0.9736842105263158
Accuracy at fold 6: 0.9649122807017544
Accuracy at fold 7: 0.956140350877193
Accuracy at fold 8: 0.9649122807017544
Accuracy at fold 9: 0.9298245614035088
Accuracy at fold 10: 0.9736842105263158
Average Accuracy =  0.963157894736842
